<a href="https://colab.research.google.com/github/seungyihong/jeju_card_data_dacon/blob/master/Card_Base_Line_Bigai_Model_FINAL07_3_ipnyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
#@title 사용할 데이터셋 및 모델 선택, 경로 설정
#@markdown /My Drive/ [ **any directory you use** ]
Directory = "\uB370\uC774\uCF58, \uC81C\uC8FC\uB3C4\uC2E0\uC6A9\uCE74\uB4DC" #@param {type:"string"}
Dataset = "toured_data2.csv" #@param {type:"string"} ['201901-202003.csv']
Programmer_Name = "Seung" #@param {type : "string"} ['Jang', 'Hyun', 'Virus', 'Seung', 'Silver' ]
Model_Name = "Seung"#@param{type: "string"}

currDir = '/content/drive/My Drive/' + Directory
datasetDir = currDir  + '/0_Data/' + Dataset

#현재 코드가 실행되는 디렉토리명
print("현재 경로:", currDir)
print("데이터셋 경로:", datasetDir)

현재 경로: /content/drive/My Drive/데이콘, 제주도신용카드
데이터셋 경로: /content/drive/My Drive/데이콘, 제주도신용카드/0_Data/toured_data2.csv


In [29]:
!pip install category_encoders

In [30]:
!pip install lightgbm

In [31]:
import sys
print('python version:',sys.version[:6])
import numpy as np
print('numpy version:',np.__version__)
import pandas as pd
print('pandas version:',pd.__version__)

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
print('seaborn version:',sns.__version__)

import time
import datetime as dt
from dateutil.parser import parse
import missingno as msno
import gc
import warnings 

from itertools import chain, permutations, combinations

import pickle

import sklearn

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split

from category_encoders import *
import multiprocessing
import lightgbm as lgb

warnings.filterwarnings('ignore')

plt.rc('font', family='NanumBarunGothic')
mpl.style.use('ggplot')

tour_data = 0

## 데이터 크기 확인 함수
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

## 파일 저장 함수
def save_it(target_p, filename_p, format_p='pickle'):
  '''파일 저장'''
  if format_p == 'pickle':
    with open(currDir + '/1_Model/' + filename_p + '.pkl', 'wb') as f:
      pickle.dump(target_p, f)
    print('---' + filename_p + '.pkl save...')

  if format_p == 'parquet':
    target_p.to_parquet(currDir + '/1_Model/' + filename_p + '.gzip', 
                       compression='gzip')
    print('---' + filename_p + '.gzip save...')

def import_dataset():
  with open(currDir + '/1_Model/csd_enc.pkl','rb') as f:
      csd_enc = pickle.load(f)
  with open(currDir + '/1_Model/std_enc.pkl','rb') as f:
      std_enc = pickle.load(f)
  with open(currDir + '/1_Model/hsd_enc.pkl','rb') as f:
      hsd_enc = pickle.load(f)
  dtype_dict = {'CARD_SIDO_NM': np.int16, 
                'STD_CLSS_NM' : np.int16, 
                'HOM_SIDO_NM': np.int16, 
                'AGE'         : np.int8, 
                'SEX_CTGO_CD' : np.int8, 
                'FLC'         : np.int8, 
                'CSTMR_CNT'   : np.int32, 
                'AMT'         : np.int64, 
                'CNT'         : np.int32,
                'YEAR'        : np.int16,
                'MONTH'       : np.int8,
                'TOUR'        : float} 
  usecols_list = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT', 'YEAR', 'MONTH', 'TOUR']
  data = pd.read_csv(currDir + '/0_Data/toured_basic_data2.csv', usecols=usecols_list, dtype=dtype_dict)
  print('최초 데이터 불러온 상태^^')
  print(data.info()); print('='*100)
  print(data.head()); print('='*100)
  return data, csd_enc, std_enc

def Onehot_Enc(Data_Frame, col_list):
  '''
    Data_Frame 과 Bin_enc가 필요한 col_list 입력시
    해당 Data_Frame의 해당 col 에 Bin_enc 적용, 해당 인코더 반환
  '''
  oe = OneHotEncoder(cols=col_list).fit(Data_Frame)
  Data_Frame = oe.transform(Data_Frame)
    
  return Data_Frame, oe

# 스케일러 정의 / AGE, SEX_CTGO_CD, YEAR, MONTH 적용
def Imply_Scaler(Data_Frame, col_, scaler = MinMaxScaler):
  '''
      Data_Frame 과 Scaler가 필요한 col_ 입력시
      해당 Data_Frame의 해당 col 에 Scaler 적용, Scaler = ['StandardScaler', 'RobustScaler', 'MinMaxScaler', 'MaxAbsScaler']
  '''
  sc = scaler()
  sc.fit(Data_Frame[col_].values.reshape(-1,1))
  Data_Frame[col_] = sc.transform(Data_Frame[col_].values.reshape(-1,1))

  return Data_Frame, sc
      
# ['CNT', 'AMT', 'CSTMR_CNT] Residual 및 log1p 처리 적용
def seung_log(input_p):
  # dataframe['feature'] = dataframe['feature'].apply(lambda x: seung_log(x)) 로 사용
  if input_p >= 0:
    result = np.log1p(input_p)
  elif input_p < 0:
    result = -np.log1p(-input_p)
  return result

def seung_exp(input_p):
  # dataframe['feature'] = dataframe['feature'].apply(lambda x: seung_exp(x)) 로 사용
  if input_p >= 0:
    result = np.exp(input_p)-1
  elif input_p < 0:
    result = -np.exp(-input_p)+1
  return result

def make_clss_df(CLASS, copy_data,tour=True, corona=True, onehot=True, scaling=True):
  CLASS_INT = std_enc.transform([CLASS])[0]
  clss_df = (copy_data[copy_data['STD_CLSS_NM'] == CLASS_INT]).reset_index(drop=True)
  tour_data = clss_df[['CARD_SIDO_NM', 'STD_CLSS_NM', 'MONTH', 'YEAR', 'TOUR']]
  tour_data = tour_data.drop_duplicates()
  copy_data = clss_df.drop('TOUR', axis=1)
  # print(copy_data); print('='*100)

  def make_res_dataset(dataframe_p, next_year, next_month):
    '''
    특정 업종 데이터셋과 필요 년월 입력시, / 2019.2~2020.3 까지 인자로 입력 가능
    해당 년월에 대한 잔차 데이터셋 구성 / y=AMT 에 대해 np.log1p 시행
    '''
    if (next_year == 2020) and (next_month == 1):
      curr_year = 2019
      curr_month = 12
    else:
      curr_year = next_year
      curr_month = next_month-1
    curr_df_ = dataframe_p[(dataframe_p['YEAR'] == curr_year ) & (dataframe_p['MONTH'] == curr_month )] # n-1 월의 데이터셋
    next_df_ = dataframe_p[(dataframe_p['YEAR'] == next_year ) & (dataframe_p['MONTH'] == next_month )] # n   월의 데이터셋
    # print(curr_df_); print('='*100)
    # print(next_df_); print('='*100)
    if len(curr_df_) == 0 :
      return ('>>>Error: 1올바른 년월을 입력해주십시오 / 2019.2~2020.3 까지 인자로 입력 가능')

    curr_df_ = curr_df_.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']).agg({'AMT':np.sum, 'CNT':np.sum, 'CSTMR_CNT':np.sum})
    next_df_ = next_df_.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']).agg({'AMT':np.sum, 'CNT':np.sum, 'CSTMR_CNT':np.sum})
    curr_df_ = curr_df_.reset_index()
    next_df_ = next_df_.reset_index()
    curr_df_['YEAR'] = curr_year; curr_df_['MONTH'] = curr_month
    next_df_['YEAR'] = next_year; next_df_['MONTH'] = next_month
    temp_df_ = pd.merge(curr_df_,
                    next_df_,
                    how = 'inner',
                    on = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC'],
                    suffixes = ('_CURR', '_NEXT'))
    # print(temp_df_); print('='*100)
    temp_df_.rename(columns = {'YEAR_NEXT': 'YEAR', 'MONTH_NEXT': 'MONTH'}, inplace=True)
    temp_df_['AMT_RES'] = temp_df_['AMT_NEXT']-temp_df_['AMT_CURR'] 
    temp_df_['CNT_RES'] = temp_df_['CNT_NEXT']-temp_df_['CNT_CURR'] 
    temp_df_['CSTMR_CNT_RES'] = temp_df_['CSTMR_CNT_NEXT']-temp_df_['CSTMR_CNT_CURR']
    
    temp_df_['AMT_RES'] = temp_df_['AMT_RES'].apply(lambda x: seung_log(x))
    temp_df_['CNT_RES'] = temp_df_['CNT_RES'].apply(lambda x: seung_log(x))
    temp_df_['CSTMR_CNT_RES'] = temp_df_['CSTMR_CNT_RES'].apply(lambda x: seung_log(x))

    temp_df_.drop(['YEAR_CURR', 'MONTH_CURR', 'AMT_NEXT', 'AMT_CURR', 'CNT_NEXT', 'CNT_CURR',
                  'CSTMR_CNT_NEXT', 'CSTMR_CNT_CURR'], axis=1, inplace=True)  
    # print(temp_df_); print('='*100)
    return temp_df_

  def make_total_res_dataset(dataframe_p, start_year, start_month, end_year, end_month):
    '''
    정해진 기간 사이에서 make_res_dataset 시행 후 하나의 데이터셋으로 구성
    '''
    
    total_res_df_ = pd.DataFrame(columns=['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD',
                                        'FLC', 'YEAR', 'MONTH', 'AMT_RES', 'CNT_RES', 'CSTMR_CNT_RES'])

    for date_ in pd.date_range(str(start_year)+'-'+str(start_month), 
                              str(end_year)+'-'+str(end_month+1),
                              freq='M'):
      print('>>>{:4}년 {:2}월 데이터 통합 진행중...'.format(date_.year, date_.month))

      res_df_ = make_res_dataset(dataframe_p, date_.year, date_.month)
      # print(res_df_); print('='*100)
      total_res_df_ = pd.merge(total_res_df_, res_df_, how='outer')
      # print(total_res_df_); print('='*100)
      # except:
      #   return ('>>>Error: 2올바른 년월을 입력해주십시오 / 2019.2~2020.3 까지 인자로 입력 가능')
    print('>>>통합 끝이지렁!')
    return total_res_df_

  res_data = make_total_res_dataset(copy_data, 2019, 2, 2020, 4)
  dtype_dict = {'CARD_SIDO_NM'  : np.int16, 
              'STD_CLSS_NM'     : np.int16, 
              'AGE'             : np.int8, 
              'SEX_CTGO_CD'     : np.int8, 
              'FLC'             : np.int8, 
              'CSTMR_CNT_RES'   : float, 
              'AMT_RES'         : float, 
              'CNT_RES'         : float,
              'YEAR'            : np.int16,
              'MONTH'           : np.int8 } 
  for (k,v) in dtype_dict.items():
    res_data[k] = res_data[k].astype(v)
  # print('res_data.info()')
  # print(res_data.info()); print('='*100)
  # print('res_data.head()')
  # print(res_data); print('='*100)
  # print('tour_data')
  # print(tour_data); print('='*100)
  if tour==True:
    # print('merge_data')
    merge_data = pd.merge(res_data, tour_data, on=['CARD_SIDO_NM', 'STD_CLSS_NM', 'MONTH', 'YEAR'], how='left') # 투어처리
    # print(merge_data); print('='*100)
    res_data = merge_data
  if corona==True:  
    # 2020-2, 3에만 CORONA = 1, 나머지 = -1 적용 / 이거 템플릿 완성하고 해야댐
    merge_data = res_data
    merge_data['CORONA'] = -1
    merge_data['CORONA'][(merge_data['YEAR'] == 2020) & (merge_data['MONTH'] == 2)] = 1
    merge_data['CORONA'][(merge_data['YEAR'] == 2020) & (merge_data['MONTH'] == 3)] = 1
    merge_data['CORONA'][(merge_data['YEAR'] == 2020) & (merge_data['MONTH'] == 4)] = 1
    # print(merge_data); print('='*100)
    res_data = merge_data

  if onehot==True:
    oe_list = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'SEX_CTGO_CD', 'FLC']
    merge_data, oe = Onehot_Enc(res_data, oe_list)
    # print(merge_data); print('='*100)
    res_data = merge_data

  if scaling==True:
    sc_list = ['AGE', 'MONTH', 'AMT_RES', 'CNT_RES', 'CSTMR_CNT_RES', 'TOUR']
    df, sc_month = Imply_Scaler(res_data, 'MONTH') 
    df, sc_amtres = Imply_Scaler(df, 'AMT_RES')
    df, sc_cntres = Imply_Scaler(df, 'CNT_RES') 
    df, sc_cstmrres = Imply_Scaler(df, 'CSTMR_CNT_RES') 
    if tour==True:
      df, sc_tour = Imply_Scaler(df, 'TOUR') 
    df, sc_age = Imply_Scaler(df, 'AGE') 
    res_data = df
 
  return res_data

def shuffle_spliter(dataframe_):
  '''
  res_df 에서 (X,y) 분할
  '''
  #dataframe_ = dataframe_p.sample(frac=1, random_state=42)
  y_ = dataframe_['AMT_RES']
  X_ = dataframe_.drop(['AMT_RES','YEAR'], axis=1)  #YEAR는 추후 다시 살려라
  return X_,y_#np.array(X_), np.array(y_)


# pred 할 X_test 넣으려고 2020 04 cnt, custom_cnt 만들고 새 데이터 만드는 과정 

def make_X_test(CLASS, copy_data,year,month,tour=True, corona=True, onehot=True, scaling=True):
  CLASS_INT = std_enc.transform([CLASS])[0]
  clss_df = (copy_data[copy_data['STD_CLSS_NM'] == CLASS_INT]).reset_index(drop=True)
  tour_data = clss_df[['CARD_SIDO_NM', 'STD_CLSS_NM', 'MONTH', 'YEAR', 'TOUR']]
  tour_data = tour_data.drop_duplicates()
  copy_data = clss_df.drop('TOUR', axis=1)

  # 2020년 4월에 대한 CNT_RES,CSTMR_CNT_RES 가 필요함

  # 2020 03 지역-업종-나이... 템플릿 틀에 맞게 2019,03,04 짱박아 넣는 과정 
  X_19_curr = clss_df[(clss_df['YEAR'] == year-1) & (clss_df['MONTH'] == month)]
  X_19_next = clss_df[(clss_df['YEAR'] == year-1) & (clss_df['MONTH'] == month+3)]
  X_20_curr = clss_df[(clss_df['YEAR'] == year) & (clss_df['MONTH'] == month)]

  # tour

  X_19_curr  = X_19_curr .groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']).agg({'CNT':np.sum, 'CSTMR_CNT':np.sum}).reset_index()
  X_19_next = X_19_next.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']).agg({'CNT':np.sum, 'CSTMR_CNT':np.sum}).reset_index()
  X_20_curr = X_20_curr.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']).agg({'CNT':np.sum, 'CSTMR_CNT':np.sum}).reset_index()

  X_1920_curr = pd.merge(X_20_curr ,
                   X_19_curr,
                   how = 'left',
                   on = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC'],
                   suffixes = ('_20_curr', '_19_curr'))

  X_20_curr_all = pd.merge(X_1920_curr,
                   X_19_next,
                   how = 'left',
                   on = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC'],
                   suffixes = ('_1920_curr', '_19_next'))

  # # 추후 Nan 값 채우는 방법 고려해봐야 할것(2019년에는 없는데 2020년에 생긴경우..등등 으로 인해 Nan 값 존재)
  X_20_curr_all = X_20_curr_all.sort_values(by=['CARD_SIDO_NM','CNT_20_curr','FLC','AGE'])
  X_20_curr_all= X_20_curr_all.fillna(method = 'bfill')
  X_20_curr_all = X_20_curr_all.fillna(method = 'ffill')
  X_20_curr_all = X_20_curr_all.sort_values(by=['CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC'])
  
  # 2019,3,4월의 증감율 계산하고 2020 03에 적용 후 2020 04 CNT,CSTMR_CNT 생성
  p_cnt = (X_20_curr_all['CNT'] - X_20_curr_all['CNT_19_curr'])/X_20_curr_all['CNT_19_curr']
  p_cstmr_cnt = (X_20_curr_all['CSTMR_CNT'] - X_20_curr_all['CSTMR_CNT_19_curr'])/X_20_curr_all['CSTMR_CNT_19_curr']
  X_20_curr_all['CNT_20_next'] = np.round(X_20_curr_all['CNT_20_curr']*(1+p_cnt))
  X_20_curr_all['CSTMR_CNT_20_next'] = np.round(np.where(X_20_curr_all['CSTMR_CNT_20_curr']*(1+p_cstmr_cnt)> X_20_curr_all['CNT_20_next'],X_20_curr_all['CNT_20_next'],X_20_curr_all['CSTMR_CNT_20_curr']*(1+p_cstmr_cnt)))
  
  X_20_curr_all = X_20_curr_all.drop(['CNT_19_curr','CSTMR_CNT_19_curr','CNT','CSTMR_CNT'],axis=1)
  X_20_curr_all['CNT_20_next'] = X_20_curr_all['CNT_20_next'].astype('int16')
  X_20_curr_all['CSTMR_CNT_20_next'] = X_20_curr_all['CSTMR_CNT_20_next'].astype('int16')

  X_20_curr_all['CNT_RES'] = X_20_curr_all['CNT_20_next']-X_20_curr_all['CNT_20_curr']
  X_20_curr_all['CSTMR_CNT_RES'] = X_20_curr_all['CSTMR_CNT_20_next']-X_20_curr_all['CSTMR_CNT_20_curr']
  X_test = X_20_curr_all.drop(['CNT_20_curr','CSTMR_CNT_20_curr','CNT_20_next','CSTMR_CNT_20_next'],axis=1)
  X_test['YEAR'] = year
  X_test['MONTH'] = month+1
  X_test = X_test.reset_index(drop=True)
  X_test = X_test.drop(['YEAR','HOM_SIDO_NM'],axis=1)
  X_test = X_test.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM','AGE','SEX_CTGO_CD', 'FLC','MONTH']).agg({'CNT_RES':np.sum, 'CSTMR_CNT_RES':np.sum}).reset_index()

  if tour==True:
    # print('merge_data')
    merge_data = pd.merge(X_test, tour_data, on=['CARD_SIDO_NM', 'STD_CLSS_NM', 'MONTH', 'YEAR'], how='left') # 투어처리
    # print(merge_data); print('='*100)
    X_test = merge_data

  if corona==True:  
    # 2020-2, 3에만 CORONA = 1, 나머지 = -1 적용 / 이거 템플릿 완성하고 해야댐
    X_test['CORONA'] = 1

  if onehot==True:
    oe_list = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'SEX_CTGO_CD', 'FLC']
    merge_data, oe = Onehot_Enc(X_test, oe_list)
    # print(merge_data); print('='*100)
    X_test = merge_data

  if scaling==True:
    sc_list = ['AGE', 'MONTH', 'CNT_RES', 'CSTMR_CNT_RES', 'TOUR']

    X_test['AGE'] = sc_age.transform(X_test['AGE'].values.reshape(-1,1))
    X_test['MONTH'] = sc_month.transform(X_test['MONTH'].values.reshape(-1,1))
    X_test['CNT_RES'] = sc_cntres.transform(X_test['CNT_RES'].values.reshape(-1,1))
    X_test['CSTMR_CNT_RES'] = sc_cstmrres.transform(X_test['CSTMR_CNT_RES'].values.reshape(-1,1))
    # if tour==True:
    #   X_test['TOUR'] = sc_cstmrres.transform(X_test['TOUR'].values.reshape(-1,1))
  
  return X_test


# X_Test 만든 값에 2019_n월 데이터 append 하는 과정 + 랜포 예측으로 cnt,cstmr_cnt 값 채워 넣기 

def plus_X_test(X_test,X_plus):

  X_plus = X_plus.drop(['CNT_RES','CSTMR_CNT_RES'],axis=1)

  # # append 할 데이터 추출 및 기본 데이터(age,flc...) 채워 넣기
  temp_ = pd.merge(X_test,X_plus, how='outer', on = ['STD_CLSS_NM', 'CARD_SIDO_NM'], suffixes=('', '_plus'))
  temp_ = temp_[pd.isna(temp_['AGE'])]
  idx = temp_.index
  if idx.shape[0] != 0:
    # return X_test, X_test.shape[0], None, None
    col_lst = ['AGE','SEX_CTGO_CD','FLC','MONTH']
    for j in col_lst:
      for i in idx:
        temp_.loc[i,j] = temp_.loc[i,j+'_plus']

    temp_ = temp_.drop(['AGE_plus','SEX_CTGO_CD_plus','FLC_plus','MONTH_plus'],axis=1)
    X_test = pd.concat([X_test,temp_]).reset_index(drop=True)

    # # 랜포 학습을 위한 데이터 다듬기 

    X_test_exist = X_test[pd.notnull(X_test['CNT_RES'])]
    X_test_nan = X_test[pd.isna(X_test['CNT_RES'])]

    y_test_cnt = X_test_exist['CNT_RES']
    y_test_cstmr = X_test_exist['CSTMR_CNT_RES']
    X_test_exist = X_test_exist.drop(['CARD_SIDO_NM','STD_CLSS_NM','MONTH','CNT_RES','CSTMR_CNT_RES'],axis=1)

    X_test_nan = X_test_nan.drop(['CARD_SIDO_NM','STD_CLSS_NM','MONTH','CNT_RES','CSTMR_CNT_RES'],axis=1)
    idx_append = X_test_nan.index

  else:
    print('===')
    y_test_cnt = X_test['CNT_RES']
    y_test_cstmr = X_test['CSTMR_CNT_RES']
    X_test_exist = X_test.drop(['CARD_SIDO_NM','STD_CLSS_NM','MONTH','CNT_RES','CSTMR_CNT_RES'],axis=1)
    idx_append = None

  # # 각 랜포 모델로, NAN으로 비어져 있는 CNT, CSTMR_CNT 각 각 예측 
  rf = RandomForestRegressor(random_state = 0,n_jobs=-1)
  rf.fit(X_test_exist,y_test_cnt)

  rf2 = RandomForestRegressor(random_state = 0,n_jobs=-1)
  rf2.fit(X_test_exist,y_test_cstmr)

  if idx.shape[0] != 0:
    cnt_pred = rf.predict(X_test_nan)
    cstmr_pred = rf2.predict(X_test_nan)

    X_test.loc[idx_append,'CNT_RES'] = cnt_pred
    X_test.loc[idx_append,'CSTMR_CNT_RES'] = cstmr_pred

  return X_test, idx_append, rf, rf2
# 예측 템플릿 만들기 

# 제출 형식 맞추기
def make_sub_template(temp,pred_ = None,y_n_1=None,merge = False):
  temp['REG_YYMM'] = int(202007) # 나중에 7월 꺼 할꺼면 202007로 고치기 
  temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']]
  if merge == True:
    temp = pd.merge(temp,pred,left_index=True,right_index=True)
    temp['AMT'] = (temp['AMT']*2) + y_n_1
  temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

  return temp

# pred = y4_y3 에 y_3 더하기 위해 y3 생성 
def make_y_n_1(CLASS,copy_data,year,month):
  CLASS_INT = std_enc.transform([CLASS])[0]
  clss_df = (copy_data[copy_data['STD_CLSS_NM'] == CLASS_INT]).reset_index(drop=True)
  copy_data = clss_df.drop('TOUR', axis=1)

  temp_ = copy_data[(copy_data['YEAR'] == year) & (copy_data['MONTH'] == month)]
  temp_ = temp_.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM','AGE', 'SEX_CTGO_CD', 'FLC']).agg({'AMT':np.sum}).reset_index()
  amt_ = temp_['AMT']
  
  return amt_

# 예측할 year,month 입력 (현재 2020,04)
def make_permu_temp(year,month,CLASS_INT):
  SEX_CTGO_CDs  = [1,2]
  FLCs          = [1,2,3,4,5]
  years         = [year]
  months        = [month]
  CARD_SIDO_NMs = list(range(0,17))
  STD_CLSS_NMs  = [CLASS_INT] 
  HOM_SIDO_NMs  = list(range(0,17))
  AGEs          = [1,2,3,4,5,6,7]

  permu_temp = []
  for CARD_SIDO_NM in CARD_SIDO_NMs:
      for STD_CLSS_NM in STD_CLSS_NMs:
          for HOM_SIDO_NM in HOM_SIDO_NMs:
              for AGE in AGEs:
                  for SEX_CTGO_CD in SEX_CTGO_CDs:
                      for FLC in FLCs:
                          for year in years:
                              for month in months:
                                  permu_temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
  permu_temp = np.array(permu_temp)
  permu_temp = pd.DataFrame(data= permu_temp)
  permu_temp.columns = ['CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE',
                  'SEX_CTGO_CD','FLC','YEAR','MONTH']
  return permu_temp


python version: 3.6.9 
numpy version: 1.18.5
pandas version: 1.0.5
seaborn version: 0.10.1


In [32]:
def make_append_set(CLASS, copy_data,year,month,tour=True, corona=True, onehot=True, scaling=True):
  CLASS_INT = std_enc.transform([CLASS])[0]
  clss_df = (copy_data[copy_data['STD_CLSS_NM'] == CLASS_INT]).reset_index(drop=True)
  tour_data = clss_df[['CARD_SIDO_NM', 'STD_CLSS_NM', 'MONTH', 'YEAR', 'TOUR']]
  tour_data = tour_data.drop_duplicates()
  copy_data = clss_df.drop('TOUR', axis=1)
 
  X_19_curr = copy_data[(copy_data['YEAR'] == year) & (copy_data['MONTH'] == month)]
  X_19_prev = copy_data[(copy_data['YEAR'] == year) & (copy_data['MONTH'] == month-3)]

  X_19_curr  = X_19_curr.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']).agg({'CNT':np.sum, 'CSTMR_CNT':np.sum}).reset_index()
  X_19_prev = X_19_prev.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']).agg({'CNT':np.sum, 'CSTMR_CNT':np.sum}).reset_index()

  X_19_curr_all  = pd.merge(X_19_curr,
                   X_19_prev,
                   how = 'left',
                   on = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC'],
                   suffixes = ('_19_curr', '_19_prev'))
  
  X_19_curr_all = X_19_curr_all.sort_values(by=['CARD_SIDO_NM','CNT_19_curr','FLC','AGE'])
  X_19_curr_all= X_19_curr_all.fillna(0)

  X_19_curr_all = X_19_curr_all.sort_values(by=['CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC'])

  X_19_curr_all['CNT_RES'] = X_19_curr_all['CNT_19_curr']-X_19_curr_all['CNT_19_prev']
  X_19_curr_all['CSTMR_CNT_RES'] = X_19_curr_all['CSTMR_CNT_19_curr']-X_19_curr_all['CSTMR_CNT_19_prev']
  X_append = X_19_curr_all.drop(['CNT_19_curr','CSTMR_CNT_19_curr','CNT_19_prev','CSTMR_CNT_19_prev'],axis=1)
  X_append ['MONTH'] = month
  X_append  = X_append .reset_index(drop=True)

  X_append = X_append.drop(['HOM_SIDO_NM'],axis=1)
  X_append = X_append.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM','AGE','SEX_CTGO_CD', 'FLC','MONTH']).agg({'CNT_RES':np.sum, 'CSTMR_CNT_RES':np.sum}).reset_index()

  if tour==True:
    # print('merge_data')
    merge_data = pd.merge(X_append, tour_data, on=['CARD_SIDO_NM', 'STD_CLSS_NM', 'MONTH', 'YEAR'], how='left') # 투어처리
    # print(merge_data); print('='*100)
    X_append = merge_data

  if corona==True:  
    # 2020-2, 3에만 CORONA = 1, 나머지 = -1 적용 / 이거 템플릿 완성하고 해야댐
    X_append['CORONA'] = 0

  if onehot==True:
    oe_list = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'SEX_CTGO_CD', 'FLC']
    merge_data, oe = Onehot_Enc(X_append, oe_list)
    # print(merge_data); print('='*100)
    X_append = merge_data

  if scaling==True:
    sc_list = ['AGE', 'MONTH', 'CNT_RES', 'CSTMR_CNT_RES', 'TOUR']
    X_append['AGE'] = sc_age.transform(X_append['AGE'].values.reshape(-1,1))
    X_append['MONTH'] = sc_month.transform(X_append['MONTH'].values.reshape(-1,1))
    X_append['CNT_RES'] = sc_cntres.transform(X_append['CNT_RES'].values.reshape(-1,1))
    X_append['CSTMR_CNT_RES'] = sc_cstmrres.transform(X_append['CSTMR_CNT_RES'].values.reshape(-1,1))

  return X_append

In [33]:
from collections import defaultdict # 나중에 셀 복붙할 사람들은 위에 append 해주세용

def extract_meaningful_value(data_p, year, month):
    '''data_p = 'data', 'clss_df',... etc. year = 2020 , month = 3, 4
    한글로 인코딩 되어 있는 경우는 ver_come에 따로 저장해놨음!!'''
    data_p['CARD_SIDO_NM'] = data_p['CARD_SIDO_NM'].astype('category')
    data_p['STD_CLSS_NM'] = data_p['STD_CLSS_NM'].astype('category')
    data_p['HOM_SIDO_NM'] = data_p['HOM_SIDO_NM'].astype('category')
    data_p['AGE'] = data_p['AGE'].astype('category')
    x = data_p[(data_p['YEAR']== year) & (data_p['MONTH']==month)]
    x = x.groupby(['YEAR','MONTH','STD_CLSS_NM','CARD_SIDO_NM'])['AMT'].sum().reset_index()
    find = x[x['AMT'].isna() == True]

    index_values = find[['STD_CLSS_NM','CARD_SIDO_NM']].sort_values(by='STD_CLSS_NM').values
    dict_values = defaultdict(list)
    for key, value in index_values:
        dict_values[key].append(value) # year, month에 해당하는 nan 값 dict 생성
    
    card_temp = data_p.groupby(['STD_CLSS_NM', 'CARD_SIDO_NM', 'YEAR','MONTH'])['AMT'].sum()
    card_temp = card_temp.reset_index()
    mask_temp = pd.DataFrame()
    for key, values in dict_values.items():
        for value in values:
            get_temp = card_temp[(card_temp['STD_CLSS_NM']==key) & (card_temp['CARD_SIDO_NM']==value)]
            mask_temp = pd.concat([mask_temp, get_temp],axis=0)

    # mask_temp에는 해당 년월 NaN값 (업종 * 시도) *24개월 ex. 2020.03월 => 87 * 24 = 2088 rows

    mask_temp['ADD'] = 1/3     
    # 분류 알고리즘 : 살릴 것 찾기
    # 1단계 : '해외 관련된 정기 항공 운송업' 및 '면세점' 삭제

    first_temp = mask_temp.drop(mask_temp[mask_temp['STD_CLSS_NM']==14].index) # 면세점 14개 삭제
    first_temp = first_temp.drop(first_temp[first_temp['STD_CLSS_NM']==30].index) # 정기 항공 운송업 12개 삭제

    # 2단계 : 2020년 2월 3월 4월 데이터가 모두 NaN값인가? 한개라도 있으면 구제 시키자.
    mask = first_temp[(first_temp['YEAR']== 2020) & ((first_temp['MONTH']==2)
                                                | (first_temp['MONTH']==3)
                                                | (first_temp['MONTH']==4))] # 2, 3, 4월 데이터 다 갖고옴
    mask['AMT'] = mask['AMT'].fillna(0)
    group_mask = mask.groupby(['STD_CLSS_NM','CARD_SIDO_NM','YEAR'])['AMT','ADD'].sum()
    group_mask = group_mask.reset_index()
    group_mask = group_mask.drop(columns=['YEAR'])
    group_mask['AMT'] = group_mask['AMT'].fillna(0)
    filtered_mask = group_mask[group_mask['AMT'] != 0] # 다 더했더니 0이 아니면 일단 살리면 됨
    unfiltered_mask = group_mask[(group_mask['ADD']== 1) & (group_mask['AMT'] == 0)] # 3단계에 넘어가기 위한 작업


    index_values = filtered_mask[['STD_CLSS_NM','CARD_SIDO_NM']].sort_values(by='STD_CLSS_NM').values
    dict_values = defaultdict(list)
    for key, value in index_values:
        dict_values[key].append(value)


    # 3단계 : 2020.02~04 기준 으로 NaN일 때, 성수기 7월같이 의미있게 살게된 값들을 구제하자.
    unfilter_values = unfiltered_mask[['STD_CLSS_NM','CARD_SIDO_NM']].sort_values(by='STD_CLSS_NM').values
    unfilter_dict_values = defaultdict(list)
    for key, value in unfilter_values:
        unfilter_dict_values[key].append(value)

    unmask_temp = pd.DataFrame()
    for key, values in unfilter_dict_values.items():
        for value in values:
            get_temp = card_temp[(card_temp['STD_CLSS_NM']==key) & (card_temp['CARD_SIDO_NM']==value)] # 2단계에 구제받지 못한자들의 전체 분포 다시 생성
            unmask_temp = pd.concat([unmask_temp, get_temp],axis=0)

    second_mask = unmask_temp[(unmask_temp['YEAR']== 2019) & ((unmask_temp['MONTH']==2)
                                                | (unmask_temp['MONTH']==3)
                                                | (unmask_temp['MONTH']==4)
                                                | (unmask_temp['MONTH']==7))] # 조건부 : 19년 2~4월 과 19년 7월의 포지션 살리기

    second_mask['isna'] = second_mask[(second_mask['MONTH']==2)&
                                    (second_mask['MONTH']==3)&
                                    (second_mask['MONTH']==4)]['AMT'].isna() * 1 # 조건부 1) 19년 2~4월이 다 죽은값 세기
    second_mask['isna'] = second_mask['AMT'].isna() * 1
    second_mask['isna2'] = second_mask[second_mask['MONTH']==7]['AMT'].isna() *1 # 조건부 2) 19년 7월이 살아있는 값 세기
    second_mask['isna2'] = second_mask['isna2'].fillna(0)
    second_mask['STD_CLSS_NM'] = pd.to_numeric(second_mask['STD_CLSS_NM'], downcast='integer')
    second_mask['CARD_SIDO_NM'] = pd.to_numeric(second_mask['CARD_SIDO_NM'], downcast='integer')
    second_mask = second_mask.groupby(['STD_CLSS_NM','CARD_SIDO_NM'])['isna', 'isna2'].sum().reset_index()
    second_mask = second_mask[(second_mask['isna']>=3) & (second_mask['isna2'] == 0)]

    index_values = second_mask[['STD_CLSS_NM','CARD_SIDO_NM']].sort_values(by='STD_CLSS_NM').values

    for key, value in index_values:
        dict_values[key].append(value)

    data_p['CARD_SIDO_NM'] = pd.to_numeric(data_p['CARD_SIDO_NM'], downcast='integer')
    data_p['STD_CLSS_NM'] = pd.to_numeric(data_p['STD_CLSS_NM'], downcast='integer')
    data_p['HOM_SIDO_NM'] =  pd.to_numeric(data_p['HOM_SIDO_NM'], downcast='integer')

    return dict_values

In [34]:
from sklearn.metrics import mean_squared_error
def rmsle(y_pred, dataset):
  y_true = dataset.get_label()

  y_ture = np.log1p(y_true)
  y_pred = np.log1p(y_pred)

  score = mean_squared_error(y_true, y_pred)

  return 'score', score, False

In [35]:
def search_base_for_append(CLASS_p, lst_append_p, target_month_p):
  # curr_month = n 기준, 2019-(n), 2019-(n-1), 2019-(n+1), 2020-(n-2) 만 탐색
  test_data = data.copy() # toured_basic_data / 전체 데이터 로드
  x = test_data[['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'YEAR', 'MONTH']] # 지역, 업종, 나이, 성별, 가구 데이터만 선별
  xx = x[x['STD_CLSS_NM']==CLASS_p] # 한 업종 가져오고
  xxx = xx[xx['CARD_SIDO_NM']==lst_append_p] # lst_append할 지역정보 가져오고

  search_date = [2019, target_month_p]
  y = xxx[(xxx['YEAR']==search_date[0]) & (xxx['MONTH']==search_date[1])]
  y['MONTH'] = target_month_p
  if len(y) != 0:
    return y
  search_date = [2019, target_month_p-1]
  y = xxx[(xxx['YEAR']==search_date[0]) & (xxx['MONTH']==search_date[1])]
  y['MONTH'] = target_month_p
  if len(y) != 0:
    return y  
  search_date = [2019, target_month_p+1]
  y = xxx[(xxx['YEAR']==search_date[0]) & (xxx['MONTH']==search_date[1])]
  y['MONTH'] = target_month_p
  if len(y) != 0:
    return y  
  search_date = [2020, target_month_p-2]
  y = xxx[(xxx['YEAR']==search_date[0]) & (xxx['MONTH']==search_date[1])]
  y['MONTH'] = target_month_p
  if len(y) != 0:
    return y
  return pd.DataFrame()
def fill_res_for_append(output_p):
  if len(output_p) == 0:
    dummy_df = pd.DataFrame(columns=['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'MONTH', 'CNT_RES', 'CSTMR_CNT_RES'])
    return dummy_df
  output_p.drop('YEAR', axis=1, inplace=True)
  output_p['CNT_RES'] = rf.predict(output_p[['AGE', 'SEX_CTGO_CD', 'FLC']]) # 순서 수정하고
  output_p['CSTMR_CNT_RES'] = rf2.predict(output_p[['AGE', 'SEX_CTGO_CD', 'FLC']])
  return output_p

In [36]:
# 여기선 내가 dict 뽑을 예정
data, csd_enc, std_enc = import_dataset()
test_data = data.copy()
extract_dict = extract_meaningful_value(test_data, 2020, 4) # 7월 확인 : 4, # 4월 3확인 : 3일
count = 0

for i in std_enc.classes_.tolist():
  lst_remove,lst_append = [],[]
  copy_data = data.copy()

  #print(std_enc.classes_)
  #print('>>>중에서 하나 선택해봠마')
  CLASS = i # CLASS = input() #
  res_df = make_clss_df(CLASS, copy_data,tour=False, corona=False, onehot=False, scaling=False)
  print('탬플릿 만들어진 꼬라지^^ = res_df')
  print(res_df.head(20)); print('='*100)
  cut_idx = res_df.shape[0]
  X, y = shuffle_spliter(res_df)

  # X_test 생성중 
  copy_data = data.copy() 
  X_test = make_X_test(CLASS, copy_data,2020,4,tour=False, corona=False, onehot=False, scaling=False)

  # X_test + 2019_n월 데이터 append 할때 + cnt,cstmr_cnt 값 랜포로 생성 

  copy_data = data.copy()
  X_1907 = make_append_set(CLASS, copy_data,2019,7,tour=False, corona=False, onehot=False, scaling=False)
  X_test,idx_append_start, rf, rf2 = plus_X_test(X_test,X_1907)

  # 검사할 때는 임의로 count = 업종 숫자 넣어서 확인 ㄱㄱ 
  lst = extract_dict[count]
  count +=1

  if lst:
    lst = set(lst)
    lst_check = set(X_test.loc[idx_append_start[0]:,'CARD_SIDO_NM'].values.tolist())
    lst_append =  list(lst - lst_check) # 그래프로 본결과 걸러내면 좋을 지역 
    lst_remove = list(lst_check - lst) # 그래프로 본결과 넣으면 좋을 지역 

  while lst_remove:
    remo = lst_remove.pop()
    X_test = X_test.drop(X_test[X_test['CARD_SIDO_NM'] == remo].index,axis=0)

  target_month = 7 # 7월은 이거만 바꿔라
  for sido_ in lst_append:
    # print(std_enc.transform([i])[0], sido_, target_month)
    output = search_base_for_append(i, sido_, target_month)
    # print(output); print('='*100)
    output = fill_res_for_append(output)
    # print(output); print('='*100)
    len_xtest = len(X_test)
    X_test = X_test.merge(output, how='outer')
    # print(X_test); print('='*100)
    # print('Append 전 X_test 길이: {}, Append 길이: {}, Append 후 X_test 길이: {}'.format(len_xtest, len(output), len(X_test)))

  # 스케일링 중 

  X_scale = pd.concat([X,X_test],axis=0)
  X_scale, sc_age = Imply_Scaler(X_scale, 'AGE')
  X_scale, sc_month  = Imply_Scaler(X_scale, 'MONTH')
  X_scale, sc_cntres = Imply_Scaler(X_scale, 'CNT_RES') 
  X_scale, sc_cstmrres = Imply_Scaler(X_scale, 'CSTMR_CNT_RES') 

  sc_amtres  = MinMaxScaler()
  sc_amtres .fit(y.values.reshape(-1,1))
  y = sc_amtres .transform(y.values.reshape(-1,1))
  y = y.flatten()
  y = pd.Series(y,name='AMT_RES')

  X = X_scale[:cut_idx]
  X_test = X_scale[cut_idx:].reset_index(drop=True)

  X_y = pd.concat([X,y],axis=1)
  X_y = X_y.sample(frac=1, random_state=42)
  X_test = X_test.sample(frac=1, random_state=42)
  y = X_y['AMT_RES']
  X = X_y.drop(['AMT_RES'], axis=1) # 스케일 끝


  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=42)
  print('X_train.shape, X_valid.shape, y_train.shape, y_valid.shape')
  print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape); print('='*100)

  train_ds = lgb.Dataset(X_train, label=y_train)
  val_ds = lgb.Dataset(X_valid, label=y_valid)
  params = {
            'learning_rate' : 0.0001,
            'boosting_type': 'gbdt',
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }
  model = lgb.train(params = params,
                  train_set = train_ds,
                  num_boost_round = 10000,
                  #feval = rmsle,
                  valid_sets = val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 1000
                )

  # model = Lasso(alpha = 0.01)
  # model.fit(X,y)

  pred = model.predict(X_test)
  pred = sc_amtres.inverse_transform(pred.reshape(-1,1))
  pred = pred.flatten()
  pred = pd.Series(pred,index=X_test.index, name='AMT')
  pred = pred.apply(lambda x: seung_exp(x))

  copy_data = data.copy()
  amt_2004 = make_y_n_1(CLASS, copy_data, year=2020, month=4)
  sub = make_sub_template(X_test, pred_ = pred, y_n_1 = amt_2004, merge = True)

  print(i,' :모델이 예측한 값:\n')
  print(sub)

  cls_int = std_enc.transform([CLASS])[0]
  sub_permu = make_permu_temp(year = 2020, month = 7, CLASS_INT = cls_int)
  sub_permu = make_sub_template(sub_permu, merge = False)

  sub_final = pd.merge(sub_permu,sub, how = 'left',on = ['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])
  sub_final['AMT'] = sub_final['AMT'].fillna(0)

  print(i,' :현재 만들어진 submission:\n')
  print(sub_final)

  sub_final = pd.concat([s_final,sub_final])
  s_final = sub_final.copy()

# 원핫은 이방법을 이용 못함 , 오직 라벨 인코딩만 댐 
sub_final['CARD_SIDO_NM'] = sub_final['CARD_SIDO_NM'].astype('int8')
sub_final['STD_CLSS_NM'] = sub_final['STD_CLSS_NM'].astype('int8')
sub_final['CARD_SIDO_NM'] = csd_enc.inverse_transform(sub_final['CARD_SIDO_NM'])
sub_final['STD_CLSS_NM'] = std_enc.inverse_transform(sub_final['STD_CLSS_NM'])
sub_final = sub_final.sort_values(['CARD_SIDO_NM','STD_CLSS_NM']).reset_index(drop=True)
print('최종 submission:')
sub_final

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
9              5           14    4  ...  14.934253  2.772589       2.564949
10             5           14    4  ... -14.523527 -2.995732      -2.890372
11             5           14    4  ...  10.921992  0.000000       0.000000
12             5           14    4  ... -15.745370 -4.343805      -4.262680
13             5           14    4  ...  13.554420  2.833213       2.639057
14             5           14    5  ... -13.355316 -1.609438      -0.693147
15             5           14    5  ... -15.619321 -4.691348      -4.718499
16             5           14    5  ...  13.516836  1.609438       1.609438
17             5           14    5  ... -16.293362 -4.882802      -4.564348
18             5           14    6  ...  14.817098  0.693147      -2.302585
19             5           14    6  ... -14.339956 -2.197225      -2.197225

[20 rows x 10 columns]
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape
(1555, 8) (275, 8) (1555,) (275,)
Training un

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202007,강원,건강보조식품 소매업,8.850278e+07
1,202007,강원,골프장 운영업,4.722671e+09
2,202007,강원,과실 및 채소 소매업,1.122208e+09
3,202007,강원,관광 민예품 및 선물용품 소매업,1.462149e+07
4,202007,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
692,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.375555e+09
693,202007,충북,한식 음식점업,1.891442e+10
694,202007,충북,호텔업,1.362875e+07
695,202007,충북,화장품 및 방향제 소매업,4.493475e+08


In [37]:
extract_dict = extract_meaningful_value(test_data, 2020, 4) 
extract_dict

defaultdict(list,
            {3: [9],
             4: [1, 10],
             8: [4, 6, 12],
             12: [7, 15, 13],
             13: [9],
             15: [0, 3, 7],
             23: [4],
             28: [15],
             35: [10]})

In [38]:
def Write_Submission(data, Dir, Model, Programmer):
    data2 = pd.read_csv(Dir + '/0_Data/'+"202004.csv")
    data2['STD_CLSS_NM']=data2['STD_CLSS_NM'].astype('category')
    data2['CARD_SIDO_NM']=data2['CARD_SIDO_NM'].astype('category')
    data2 = data2.groupby(["REG_YYMM","CARD_SIDO_NM","STD_CLSS_NM"])["AMT"].sum().reset_index()
    data = pd.concat([data2,data], axis=0)

    submission_directory = Dir + '/2_Submission/'+ str(round(time.time()))+'_' + Model +'_' + Programmer + '.csv'
    print(submission_directory)
    read_submission = Dir + '/0_Data/' + 'submission.csv'

    submission = pd.read_csv(read_submission, index_col=0)
    submission = submission.drop(data.columns.tolist()[-1], axis=1)
    submission = submission.merge(data, left_on=data.columns.tolist()[:-1], right_on=data.columns.tolist()[:-1], how='left')
    submission.index.name = 'id'
    submission['AMT'] = submission['AMT'].fillna(0)
    submission.to_csv(submission_directory, encoding='utf-8-sig')
    print(submission.head())

# Model_Name = 'lgb_ff'
# Programmer_Name = 'WE_JJS'
# Write_Submission(data = sub_final, Dir=currDir, Model=Model_Name, Programmer=Programmer_Name)

In [39]:
sub_final

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202007,강원,건강보조식품 소매업,8.850278e+07
1,202007,강원,골프장 운영업,4.722671e+09
2,202007,강원,과실 및 채소 소매업,1.122208e+09
3,202007,강원,관광 민예품 및 선물용품 소매업,1.462149e+07
4,202007,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
692,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.375555e+09
693,202007,충북,한식 음식점업,1.891442e+10
694,202007,충북,호텔업,1.362875e+07
695,202007,충북,화장품 및 방향제 소매업,4.493475e+08
